# Import Modules

In [7]:
import numpy as np
import pandas as pd
import torch

import utils

# Load Data

In [8]:
target_vegetable = utils.VEGETABLES[13]

In [9]:
train_test = pd.read_csv("./data/mapped_train_test.csv")
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y-%m-%d")
weather = pd.read_csv("./data/sorted_mapped_adjusted_weather.csv")
train_test = pd.concat([train_test, weather], axis=1)
train_test = train_test.reset_index(drop=True)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

# train_test["weekday"] = train_test.date.dt.weekday
# weekdays = pd.get_dummies(train_test["weekday"])
# train_test = train_test.drop(columns="weekday")
# train_test = pd.concat([train_test, weekdays], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

test_size = pd.read_csv("./data/test.csv").shape[0]
train = train_test[:-test_size]
test = train_test[-test_size:]

train = utils.get_target_values(train, target_vegetable)
test = utils.get_target_values(test, target_vegetable)

train_loader, train, test, ss = utils.preprocess_data_submit(train, test)

# Set Free Params

In [10]:
future = test.shape[0]
num_epochs = 70
learning_rate = 0.001
weight_decay = 1e-3

# Training

In [11]:
model = utils.pipeline_rnn_submit(train_loader, train, test, future=future, num_epochs=num_epochs,
                                  lr=learning_rate, weight_decay=weight_decay)

# Predict

In [17]:
with torch.no_grad():
    model.eval()
    pred_y = model.predict(train, test, future)

pred_y = pred_y.cpu().detach().numpy()
pred_y = np.concatenate([pred_y.reshape(-1, 1), test[:, 1:]], axis=1)
pred_y[:, :6] = ss.inverse_transform(pred_y[:, :6])

print(f"std: {np.std(pred_y[:, 0].tolist())}")
pred_y[:, 0].tolist()

std: 112.81506459243768


[2668.7397532812415,
 2727.713896289672,
 2783.135576950746,
 2819.4121909722353,
 2855.5540692514787,
 2894.1169874995267,
 2926.012342057659,
 3009.962706732599,
 3053.5201441342238,
 3064.178544155805,
 3072.296126481566,
 3049.6300377279013,
 3018.726437447888,
 3009.910940639387,
 2987.1216660033374,
 2960.617123331385,
 2971.387954611743,
 3022.0842278539994,
 2992.4991292809896,
 2962.4687741432444]